In [1]:
import numpy as np
import random as rn
from math import *

class RVar_Game():
    
    def __init__(self, game, defn = {}):
        
        self._GAME = game 
        self._players = self._GAME.shape[-1]
        self._choices = np.array([self._GAME.shape[i] for i in range(self._players)])
        self._defn = defn
        self._curval = {} #current values
    
    def payoff_i(self, i, action):
        # Payoff of player i with this action, everyone else fixed
        #     and last's player's action max_ind (since it's not a part of the indices)
        return self._game[tuple(self._indices[:i]) + (action,) + tuple(self._indices[(i+1):]) + (self._max_ind,)][i]
    
    def instance_generator(self):
        values = {}
        for rv in self._defn:
            values[rv] = self._defn[rv][0](*self._defn[rv][1])

        self._curval = values

    def game_generator(self):
        
        self._iterator = [0]*self._players
        num_iter = self._players
        game = np.copy(self._GAME)
        self.instance_generator()
        values = self._curval
        done = 0
        while not done:
            
            partition = game[tuple(self._iterator)]
            V = np.zeros(self._players)
            
            for i in range(len(partition)):
                if partition[i] in values:
                    V[i] = values[partition[i]]
                else:
                    V[i] = partition[i]
                    
            game[tuple(self._iterator)] = V
            # Move to a different node
            self._iterator[-1] += 1
            for i in range(num_iter-1,-1,-1):
                if self._iterator[i] == self._choices[i]:
                    if num_iter == 1:
                        done = 1  
                    else:
                        self._iterator[i] = 0
                        self._iterator[i-1] += 1
                        if self._iterator[0] == self._choices[0]:
                            done = 1
                            
        return game
             
                            
    def nash_equil(self):
                
                
        # If there are random variables, get an instance of a game
        if len(self._defn) != 0:
            self._game = self.game_generator()     
        else:
            self._game = self._GAME
        
        # Need to iterate over all players but the last one to go,
        #     could be more efficient if the players with the least
        #     number of choices were used, but right now
        #     it uses the first n-1 of n players
        
        # Assuming the first n-1 of n players choose a certain action,
        #     and the last player chooses their maximum payoff,
        #     if any of the first n-1 players change their action there is
        #     no equilibrium at the coordinates given by those action coordinates.
        # If they don't switch, there is an equilibrium precisely at the
        #     coordinates of the assumed position and the maximal payout of
        #     the nth player.
        self._indices = [0]*(self._players-1)
        num_index = self._players-1
        equilibriums = np.zeros(self._game.shape[:-1])
        done = 0
        
        while not done:
            
            partition = self._game[tuple(self._indices)]
            payoffs = [partition[i][-1] for i in range(self._choices[-1])]
            
            self._max_ind = 0
            max_payoff = payoffs[self._max_ind]
            for i in range(1,self._choices[-1]):
                if payoffs[i] > max_payoff: #Nash equilibrium with indifference (equality) or no?
                    max_payoff = payoffs[i]
                    self._max_ind = i
            
            # We've found the choice for the last player, now do the other players change what they do?
            # Go over all players and each action, for each player find maximal payoff
            #     if the action required is not the assumed at any point, break and say there is no
            #     equilibrium. If the actions match the current index there is an equilibrium.
            test_index = np.zeros(num_index)
            equil_bool = True
            for player in range(num_index):
                max_ind_i = 0
                max_payoff_i = self.payoff_i(player, max_ind_i)
                for i in range(1,self._choices[player]):
                    x = self.payoff_i(player, i)
                    if x > max_payoff_i:
                        max_payoff_i = x
                        max_ind_i = i
                        
                test_index[player] = max_ind_i
                if max_ind_i != self._indices[player]:
                    equil_bool = False
                    break
            
            if equil_bool:
                equilibriums[tuple(self._indices) + (self._max_ind,)] = 1
                
            # Go up a decision coordinate.
            self._indices[-1] += 1
            # Go over the indices from the last to the first,
            #     check if any need to be adjusted.
            for i in range(num_index-1,-1,-1):
                
                if self._indices[i] == self._choices[i]:
                    if num_index == 1:
                        # Quick fix for a special-case.
                        done = 1  
                    else:
                        self._indices[i] = 0
                        self._indices[i-1] += 1
                        if self._indices[0] == self._choices[0]:
                            # Done when the first index is greater than the number
                            #     of decision coordinates the first player has.
                            done = 1

        return equilibriums

## Now to test some notable games in the field

In [2]:
# The game profile should be read as follows:
#    the first chunk of the array is the state which is inevitable if the first player
#    commits to the first action
#    the second chunk is the one which the first player commits to the second action
# Within the chunks the process follows for each subsequent player until reaching a tuple of values
#    these values describe the payoffs in order from (first, second, ...) player

print("A non-variant prisoner's dilemma:\n")

prisoners_dilemma = np.array([[(-1,-1), (-10,0)],[(0,-10), (-8,-8)]])
A = RVar_Game(prisoners_dilemma)
print(A._GAME)
print('\nThe nash equilibrium occurs at: ')
print(A.nash_equil())
print('\nWhere both prisoners tragically suffer -8 rather than a possible -1')

A non-variant prisoner's dilemma:

[[[ -1  -1]
  [-10   0]]

 [[  0 -10]
  [ -8  -8]]]

The nash equilibrium occurs at: 
[[0. 0.]
 [0. 1.]]

Where both prisoners tragically suffer -8 rather than a possible -1


In [3]:
print("A prisoner's dilemma in which the prisoners behave variably: ")

prisoners_dilemma = np.array([[(-1,-1), ('X',0)],[(0,'Y'), (-8,-8)]], dtype = 'object')
defn = {'X' : (rn.gauss, (-5,2)), 'Y' : (rn.gauss, (-5,4))}

B = RVar_Game(prisoners_dilemma, defn)

# Keep track of how many times an equilibrium occurs at any node
sum_ = np.zeros(prisoners_dilemma[:-1].shape)
n = 10000

for _ in range (n):
    sum_ += B.nash_equil()
    
nash_probability = sum_/n
nash_invert = np.ones(prisoners_dilemma[:-1].shape)-nash_probability
nash_variance = np.multiply(nash_probability,nash_invert)/n
nash_stdev = np.sqrt(nash_variance)

print('The probability of nash equlibrium at any node:\n', nash_probability)
print('\nThe standard deviation of the probability at any node:\n', nash_stdev)

A prisoner's dilemma in which the prisoners behave variably: 
The probability of nash equlibrium at any node:
 [[[0.     0.9359]
  [0.7706 0.0135]]]

The standard deviation of the probability at any node:
 [[[0.         0.00244931]
  [0.00420447 0.00115403]]]


In [4]:
# Defining random variables before-hand

def rand_times_3_minus_x(x):
    return rn.random()*3-x

x = 1

battle_of_the_sexes = np.array([[('X','Y'), (0,0)],[(0,0), ('Y','X')]], dtype = 'object')
defn = {'X' : (rand_times_3_minus_x,(x,)), 'Y' : (rand_times_3_minus_x,(x,))}

F = RVar_Game(battle_of_the_sexes, defn)

sum_ = np.zeros(battle_of_the_sexes[:-1].shape)
n = 10000

for _ in range (n):
    sum_ += F.nash_equil()
    
nash_probability = sum_/n
nash_invert = np.ones(battle_of_the_sexes[:-1].shape)-nash_probability
nash_variance = np.multiply(nash_probability,nash_invert)/n
nash_stdev = np.sqrt(nash_variance)

print('The probability of nash equlibrium at any node:\n', nash_probability)
print('\nThe standard deviation of the probability at any node:\n', nash_stdev)

The probability of nash equlibrium at any node:
 [[[0.4435 0.332 ]
  [0.3385 0.4435]]]

The standard deviation of the probability at any node:
 [[[0.00496797 0.00470931]
  [0.00473199 0.00496797]]]


In [5]:
# There is a head on show off car collission between two players, action 1 is collide action 2 is swerve
# If they both collide they suffer -100 by losing their lives
# If player one collides but player 2 swerves player 1 is not a chicken
#    and he receives positive A as a recognition, while player two receives negative Y for being a chicken
#    but atleast he gets to keep his life
# If they both swerve neither gets recognition and both get to keep their lives, resulting in 0 

chicken = np.array([[(-100,-100), ('A','Y')],[('B','Z'),(0,0)]], dtype = 'object')
defn = {'A' : (rn.gauss, (15, 5)), 'B' : (rn.gauss, (-15, 5)), 'Z' : (rn.gauss, (23, 2)), 'Y' : (rn.gauss, (-23,2))}

G = RVar_Game(chicken, defn)

sum_ = np.zeros(chicken[:-1].shape)
n = 10000

for _ in range (n):
    sum_ += G.nash_equil()
    
nash_probability = sum_/n
nash_invert = np.ones(chicken[:-1].shape)-nash_probability
nash_variance = np.multiply(nash_probability,nash_invert)/n
nash_stdev = np.sqrt(nash_variance)

print('The probability of nash equlibrium at any node:\n', nash_probability)
print('\nThe standard deviation of the probability at any node:\n', nash_stdev)

The probability of nash equlibrium at any node:
 [[[0.    0.999]
  [1.    0.   ]]]

The standard deviation of the probability at any node:
 [[[0.         0.00031607]
  [0.         0.        ]]]
